KoBert 불러오기

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-tb81ejlr
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-tb81ejlr
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 704.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 880.5/881.9 MB 134.3 MB/s eta 0:00

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm.notebook import tqdm

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 학습했던 모델을 불러오는 과정입니다

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
tok.vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)

In [ ]:
# model.state_dict() 로 저장한 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/이창대_kaggle/11st_shopping.pt'))

<All keys matched successfully>

# Model

In [ ]:
from pprint import pprint

In [ ]:
# 불러온 모델
pprint(model, width= 200)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, 0]
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_loader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_loader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정적")
            else:
                test_eval.append("긍정적")

        print(">> 해당 리뷰는 " + test_eval[0] + " 리뷰 입니다.")

In [ ]:
max_len = 90 # 3사분위수인 90
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
while True:
    sentence = input("긍부정을 판단할 리뷰를 입력해주세요 : ")
    if sentence == "0":
      print(">> 긍부정 판단을 종료합니다!")
      break
    predict(sentence)
    print("\n")

긍부정을 판단할 리뷰를 입력해주세요 : 배송 빠르고 저렴하게 구입했어요 짱짱해서 여름에 자주 빨아 입혀도 깔끔할거 같아요 색깔도 예쁘고 마음에 들어요


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 해당 리뷰는 긍정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 손매길이가 짧아요 9부로 입어야할듯요재잴도 별로입니다
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 아니 지퍼 꼴이 왜이런거죠...지퍼 수선집에맡겨야겠어요.. 지퍼가 안올라가고 안내려가요 ㅋㅋㅋ 스커트랑 바지 샀는데 지퍼꼬라지 진심... 하..안입은상태에서 지퍼 올렸다내렸다했어요 ㅋㅋㅋ
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 0
>> 긍부정 판단을 종료합니다!


# 긍부정 연관 단어 탐구
파인튜닝한 KoBERT 모델에서 단어가 긍/부정에 어떤 영향을 미치는지 알아보려고 합니다. 

---
단어를 임베딩한 가중치(8002*768 행렬)와 마지막 Linear Layer의 가중치(768*2 행렬)과의 행렬곱으로 구하려고 합니다.



In [ ]:
weights = model.state_dict()

In [ ]:
# 파인 튜닝된 BERT parameter 값
# 12 transformer block, 768 hidden size, 12 self-attention heads 로 구성됨
list(model.state_dict().keys())

['bert.embeddings.position_ids',
 'bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias',
 'bert.encoder.layer.0.attention.self.query.weight',
 'bert.encoder.layer.0.attention.self.query.bias',
 'bert.encoder.layer.0.attention.self.key.weight',
 'bert.encoder.layer.0.attention.self.key.bias',
 'bert.encoder.layer.0.attention.self.value.weight',
 'bert.encoder.layer.0.attention.self.value.bias',
 'bert.encoder.layer.0.attention.output.dense.weight',
 'bert.encoder.layer.0.attention.output.dense.bias',
 'bert.encoder.layer.0.attention.output.LayerNorm.weight',
 'bert.encoder.layer.0.attention.output.LayerNorm.bias',
 'bert.encoder.layer.0.intermediate.dense.weight',
 'bert.encoder.layer.0.intermediate.dense.bias',
 'bert.encoder.layer.0.output.dense.weight',
 'bert.encoder.layer.0.output.dense.bias',
 'bert.encoder.layer.0.output.LayerN

In [ ]:
len(model.state_dict()['classifier.weight'][0]), len(model.state_dict()['classifier.weight'][1])

(768, 768)

In [ ]:
len(model.state_dict()['bert.embeddings.word_embeddings.weight'])

8002

In [ ]:
len(model.state_dict()['bert.embeddings.word_embeddings.weight'][0])

768

In [ ]:
weights = model.state_dict()["classifier.weight"]
word_embeddings = model.state_dict()["bert.embeddings.word_embeddings.weight"]

In [ ]:
word_embeddings.shape, weights.shape

(torch.Size([8002, 768]), torch.Size([2, 768]))

In [ ]:
# 부정의 weight
weights[0].shape

torch.Size([768])

In [ ]:
# 긍정의 weight
weights[1].shape

torch.Size([768])

In [ ]:
# word_embeddings 에 weights를 transpose 한 값을 곱해(행렬곱) score를 구함
word_score = word_embeddings.matmul(weights.T)
word_score

tensor([[ 0.0004, -0.0302],
        [ 0.0073,  0.0791],
        [-0.0043,  0.0120],
        ...,
        [ 0.0169,  0.0142],
        [ 0.0223, -0.0044],
        [ 0.0119, -0.0352]], device='cuda:0')

In [ ]:
# KoBERT에서 사용한 8002개의 단어와 이에 대한 긍/부정 점수를 확인할 수 있다.
word_score.shape

torch.Size([8002, 2])

In [ ]:
# KoBERT vocab에서 단어 가져오기
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
word_list = vocab.idx_to_token
print(len(word_list))

8002


In [ ]:
print(word_list, end='')

['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]', '!', "!'", '!”', '"', '#', '$', '%', '%)', '&', '&#34;', "'", "'(", "',", '(', '(0', '(1', '(10', '(12', '(15', '(17', '(18', '(19', '(2', '(20', '(23', '(24', '(25', '(3', '(4', '(5', '(6', '(7', '(8', '(9', '(?)', ')', ")'", '),', ')’', '*', '+', ',', '-', '----------------', '-1', '-2', '-20', '-3', '-4', '.', '...', '..."', "...'", '...”', '/', '0', '0%', '0%)', '0.0', '0.00', '0.1', '0.1%', '0.2%', '0.3', '0.3%', '0.4%', '0.5', '0.5%', '0.6', '0.6%', '0.7', '0.7%', '0.8', '0.8%', '00', '000.0', '00000', '01', '02', '02-', '03', '04', '05', '06', '07', '08', '09', '0:00:00', '1', '1%', '1%)', '1)', '1,000', '1.3%', '1.4%', '1.5%', '1.6', '1.6%', '1.7%', '1.8', '10', '100', '1000', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1⁄2', '1⁄4', '2', '2%', '2)', '2.0', '2.3%', '2.5', '2.5%', '2.8%', '20', '200', '2000', '2011', '2012', '2013', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '3%', '30', '300', '3000', '31',

In [ ]:
word_score_np = word_score.cpu().numpy()

In [ ]:
pd.DataFrame(word_score_np)

,0,1
0,0.000371,-0.030243
1,0.007252,0.079079
2,-0.004254,0.012008
3,0.022834,-0.002270
4,-0.058089,0.004298
...,...,...
7997,0.026606,0.015584
7998,-0.012013,-0.009740
7999,0.016859,0.014154
8000,0.022297,-0.004414


In [ ]:
pd.DataFrame(word_list)

,0
0,[UNK]
1,[PAD]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
7997,힌
7998,힐
7999,힐링캠프
8000,힘


In [ ]:
sentiment_insight = pd.concat([pd.DataFrame(word_list),pd.DataFrame(word_score_np)], axis=1)

In [ ]:
sentiment_insight.columns = ['단어', '부정','긍정']

# 아래와 같이 각 단어와 부정, 긍정 점수를 얻을 수 있다.

In [ ]:
sentiment_insight

,단어,부정,긍정
0,[UNK],0.000371,-0.030243
1,[PAD],0.007252,0.079079
2,[CLS],-0.004254,0.012008
3,[SEP],0.022834,-0.002270
4,[MASK],-0.058089,0.004298
...,...,...,...
7997,힌,0.026606,0.015584
7998,힐,-0.012013,-0.009740
7999,힐링캠프,0.016859,0.014154
8000,힘,0.022297,-0.004414


In [ ]:
sentiment_insight.sort_values('부정')

,단어,부정,긍정
7442,첼,-0.118954,-0.030333
7677,틸,-0.110800,-0.028070
5948,디스크,-0.096814,-0.001517
7591,탈,-0.095603,0.016188
3751,▁이하,-0.093674,0.028851
...,...,...,...
6449,빅,0.107193,0.036485
5489,교사,0.107742,-0.029648
414,j,0.108126,-0.097337
7653,퉈,0.111610,-0.031215


이 중 일부분을 살펴보면 다음과 같다. 다만, 이 점수를 통해서 명확히 무언가를 파악하기에는 어려워 보인다.

In [ ]:
sentiment_insight[(sentiment_insight['단어'] == '▁사랑을')|
                  (sentiment_insight['단어'] == '▁없')|
                  (sentiment_insight['단어'] == '▁에너지')|
                  (sentiment_insight['단어'] == '▁이벤트')|
                  (sentiment_insight['단어'] == '▁이상의')|
                  (sentiment_insight['단어'] == '▁많아')|
                  (sentiment_insight['단어'] == '▁명령')|
                  (sentiment_insight['단어'] == '▁나쁜')|
                  (sentiment_insight['단어'] == '▁할인')
                  ]

,단어,부정,긍정
1380,▁나쁜,0.008308,-0.049664
1952,▁많아,-0.043824,0.004251
2036,▁명령,0.034890,-0.003139
2591,▁사랑을,-0.008165,0.022978
3270,▁없,0.000439,0.000156
3285,▁에너지,0.040282,0.043092
3699,▁이벤트,-0.005541,0.003471
3705,▁이상의,-0.005852,0.032736
4981,▁할인,-0.006949,-0.031965


In [ ]:
# csv로 저장
sentiment_insight.to_csv('11번가_감정점수.csv', encoding='utf-8-sig')

# 감정사전을 통한 긍부정 분류

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('/content/drive/MyDrive/이창대_kaggle/11번가_리뷰(43만).csv', encoding='utf-8-sig', index_col=0)

In [10]:
# https://github.com/park1200656/KnuSentiLex
knu = pd.read_csv('/content/drive/MyDrive/이창대_kaggle/SentiWord_Dict.csv', encoding='utf-8',header=None)

In [11]:
knu.columns = ['word','score']

In [12]:
knu

,word,score
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1
...,...,...
14850,갈등,-1
14851,의혹,-1
14852,내팽개치다,-2
14853,횡령,-2


In [13]:
# 감정 점수 계산하는 코드
# 점수와 그 점수에 영향미친 단어로 계산함
# 데이터프레임으로 가져와서 numpy 형태라 np.str 사용했고 그래서 워닝 뜸

def sentiment_score(input_str):
    score = 0
    senti_word = []
    for i in range(len(knu)):
        if knu['word'][i] in input_str:
            score += knu['score'][i]
            senti_word.append(knu['word'][i] + ' ' + np.str(knu['score'][i]))
        else:
            continue
    return score, senti_word

In [14]:
sentiment_score(df['리뷰'][1])

<ipython-input-13-8185aef2ece9>:11: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  senti_word.append(knu['word'][i] + ' ' + np.str(knu['score'][i]))


(-7, ['못 0', '믿고 1', '비싼 -2', '정 1', '탈 -1', '탈이 -1', '해 -2', '허접 -2', 'ㅜ -1'])

In [15]:
review_score = []
review_score_exp = []

i = 0
for review in df['리뷰']:
    score, senti_word = sentiment_score(review)
    review_score.append(score)
    review_score_exp.append(senti_word)
    
    i+=1
    
    if i % 500 == 0:
        print(i,"번째 진행중")
    
    # 2000개에 대해서만 샘플로 진행해봄
    if i == 2000:
        break

<ipython-input-13-8185aef2ece9>:11: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  senti_word.append(knu['word'][i] + ' ' + np.str(knu['score'][i]))


500 번째 진행중
1000 번째 진행중
1500 번째 진행중
2000 번째 진행중


In [16]:
len(review_score)

2000

In [24]:
df_sentiment_dict = pd.DataFrame([review_score, review_score_exp, df['리뷰'][:2000], df['평점'][:2000]]).transpose()

In [25]:
df_sentiment_dict.columns = ('score','explanation','review', 'star')

In [29]:
df_sentiment_dict.sample(10)

,score,explanation,review,star
1781,-1,[ㅡㅡ -1],질이.안조아여.ㅡㅡㅡ별루임,1
1636,-1,[ㅠ -1],세탁하고 나니 형체가 없어졌어요ㅠㅠ빨지말고 입어야할듯~,1
1666,0,"[정 1, ㅜ -1]",헐...오트밀 샀는데 다 비쳐요... 세상에 주머니도 비칠정도니 말 다했네요. 배송...,1
1851,-2,"[; 1, 성 0, 성이 0, 져서 -1, 찢어져 -1, ㅡㅡ -1]",바지 신축성이 전혀 없어요~오늘 입구나갔는데 가르쟁이(??)가 찢어져서 담요를 두르...,1
457,4,"[마음대로 0, 좋아 2, 친절 2]",물건은 아주마음에들고 좋아요 다만 판매자보다 11번가상담원들이 불친절하고응대에미흡하...,1
553,-5,"[못 0, 비싼 -2, 없어서 -1, 함께 1, 해 -2, ㅠ -1]",비싼돈주고 20~25마리 2kg이나 주문해서 기다렸더니~~ 배송건이 많아서 이틀이나...,1
1117,-2,"[깝 -1, 아깝다 -1]",이렇게 얇은 비닐이 있을수있나요물건을담을수나있을지돈아깝다는 생각듭니다,1
244,-1,"[실망 -2, 좋아 2, ㅠ -1]",저렴하게 구입했다고 좋아했는데 재고로 만원에 판매했었던건지 만원택이 붙어있네요ㅠ 거...,1
829,-6,"[부족 -1, 부족한 -2, 상하게 -2, 이상하게 -1]",수산대전 쿠폰때문인가..이상하게 양이 20퍼 부족한 느낌이네요..,1
772,-2,"[우는 -1, 져서 -1]","매년 여기서 새우를 구매하지만, 이번 새우는 껍질이 흐물거리는 새우가 반이상 이어서...",1


In [20]:
df_sentiment_dict.to_csv('df_sentiment.csv',encoding='utf-8-sig')